# Выбор локации для скважины

В данном проекте мы будем помогать добывающей компании «ГлавРосГосНефть» с выбором региона, в котором бурить новые скважины.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Для начала изучим данные и проведем их подготовку: проверим на пропуски и дубликаты, и при необходимости исправим. 

После этого разобьем данных регионов на тренировочную и тестовую выборки и создадим модель линейной регрессии для предсказания объема нефти в скважинах. 

После этого проведем моделирование и расчитаем, какую прибыль нам может принести каждый из регионов, и какие риски при этом будут. 

После всех расчетов, проанализируем значения и выберем наиболее экономически выгодных регион для бурения.

## Загрузка и подготовка данных

### Загрузка данных

In [1]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

RANDOM_STATE = 42
TEST_SIZE = 0.75
state = np.random.RandomState(12345)

In [2]:
try:
    geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    geo_data_0 = pd.read_csv('C/datasets/geo_data_0.csv')
    geo_data_1 = pd.read_csv('C/datasets/geo_data_1.csv')
    geo_data_2 = pd.read_csv('C/datasets/geo_data_2.csv')   
    

Посмотрим на наши датафреймы.

In [3]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
geo_data_0.iloc[:, 1:].corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [5]:
geo_data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
geo_data_1.iloc[:, 1:].corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [8]:
geo_data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [9]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
geo_data_2.iloc[:, 1:].corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


In [11]:
geo_data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Данные в таблицах соответствуют описанию. Колонки в датафреймах правильно разделены, названия колонок написаны в змеином регистре, числовые колонки представлены числами. Датафреймы одинакового размера: по 100000 строк в каждом. Корреляция между входными признаками практически отсутствует.

### Предобработка данных

Посмотрим, есть ли пропуски в наших датафреймах.

In [12]:
geo_data_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [13]:
geo_data_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [14]:
geo_data_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Пропуски отсутсвтуют.

Теперь проверим наши таблицы на наличие дубликатов.

In [15]:
geo_data_0.duplicated().sum()

0

In [16]:
geo_data_1.duplicated().sum()

0

In [17]:
geo_data_2.duplicated().sum()

0

Явные дубликаты отсутствуют.

Теперь посмотрим, есть ли среди наших скважин повторяющиеся - то есть строки с одинаковыми значениями в колонке 'id'. И если такие есть, то удалим такие строки, оставив лишь последнюю встречающуюся.

In [18]:
geo_data_0.duplicated(subset='id').sum()

10

In [19]:
geo_data_1.duplicated(subset='id').sum()

4

In [20]:
geo_data_2.duplicated(subset='id').sum()

4

In [21]:
geo_data_0.drop_duplicates(subset='id', keep='last', inplace=True)
geo_data_1.drop_duplicates(subset='id', keep='last', inplace=True)
geo_data_2.drop_duplicates(subset='id', keep='last', inplace=True)

Неявные дубликаты удалены.

Теперь сделаем индексом колонку с id скважин.

In [22]:
geo_data_0 = geo_data_0.reset_index(drop=True)
geo_data_1 = geo_data_1.reset_index(drop=True)
geo_data_2 = geo_data_2.reset_index(drop=True)

geo_data_0.set_index('id', inplace=True)
geo_data_1.set_index('id', inplace=True)
geo_data_2.set_index('id', inplace=True)

### Вывод

Данные успешно загружены. Из каждого датафрейма было удалены неявные дубликаты. Они могли возникнуть из-за того, что характеристики некоторых скважин мерили несколько раз. Поэтому были оставлены лишь последние встречающиеся значения повторящихся скважин. Колонки с id скважин были сделаны инексами для удобства в дальнейшей работе.

## Обучение и проверка модели

### Создание функции для предсказания

В качестве модели предскзаания будем использовать модель линейной регрессии, так как все признаки, в том числе и целовой у нас непрерывные числа. В качестве тренировочной выборки будем использовать 75% данных, а в качестве валидационной - оставшиеся 25%.
Напишем функцию, которая для каждого региона будет разделять данные на выборки. После этого обучать модель, делать предсказание и высчитывать среднеквадратичную ошибку.

In [23]:
def prediction(data):
    X_train, X_valid, y_train, y_valid = train_test_split(
        data.drop(['id', 'product'], axis=1),
        data['product'],
        test_size = TEST_SIZE, 
        random_state = RANDOM_STATE)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    df_pred = pd.DataFrame(y_valid)
    df_pred['y_pred'] = y_pred
    rmse = mean_squared_error(y_valid, y_pred) ** 0.5
    return y_pred, rmse, df_pred

In [24]:
y_pred_0, rmse_0, df_pred_0 =  prediction(geo_data_0)
y_pred_1, rmse_1, df_pred_1 =  prediction(geo_data_1)
y_pred_2, rmse_2, df_pred_2 =  prediction(geo_data_2)

### Результаты

In [25]:
print('Средний запас предсказанного сырья в 1 регионе:', y_pred_0.mean())
print('RMSE модели:', rmse_0)
print()
print('Средний запас предсказанного сырья во 2 регионе:', y_pred_1.mean())
print('RMSE модели:', rmse_1)
print()
print('Средний запас предсказанного сырья в 3 регионе:', y_pred_2.mean())
print('RMSE модели:', rmse_2)

Средний запас предсказанного сырья в 1 регионе: 92.99989216855128
RMSE модели: 37.70274696966289

Средний запас предсказанного сырья во 2 регионе: 68.90262191309118
RMSE модели: 0.8905400869521801

Средний запас предсказанного сырья в 3 регионе: 95.01673763271201
RMSE модели: 40.015930218197234


### Вывод

Хуже всего обстоят дела во 2 регионе: средний запас предсказанного сырья в скважине составляет 68.9 тысяч баррелей. При этом уровень точности модели достаточно высок и RMSE составляет меньше 1. 

В 1 и 3 же регионе дела куда лучше: средний запас предсказанного сырья в скважине составляет 93 и 95 тысяч баррелей соотвественно. При этом RMSE уже гораздо выше: около 38 в 1 регионе, и около 40 в 3. То есть могут быть скважины, где переваливает за 100 баррелей, но есть и такие, где объем нефти будет порядка 60. Но даже такой низкий объем сопоставим со значениями во 2 регионе. Поэтому имеет смысл рассматривать 1 и 3 регионы для добычи нефти. 

Но так как нам нужно выбрать лишь 1 регион, то логично выбрать регион с наибольшим средним, т.е. 3 регион.

## Подготовка к расчёту прибыли

Запишем все ключевые значения в отдельных переменных. А именно: количество скважен для измерений - 500 штук, количество лучших скважин для дальнейшей разработки - 200 штук. Бюджет, выделенный на разработку - 10 млрд. рублей. И доход с каждый единицы (тыс. баррелей) продукта - 450000 рублей.

In [26]:
N_WELLS = 500
N_BEST_WELLS = 200
BUDGET = 10_000_000_000
REVENUE_PER_BARREL = 450_000

Так как мы знаем выделенных бюджет и количество скважин, которые будем разрабатывать, можем посчитать затраты на 1 скважину

In [27]:
cost_per_well = BUDGET / N_BEST_WELLS
print(f'Затраты на одну скважину: {cost_per_well / 1000000} млн. рублей')

Затраты на одну скважину: 50.0 млн. рублей


Так как нам известна стоимость единицы продукции, то мы можем посчитать, какое количество единиц нам нужно добыть, чтобы разработка скважины не ушла в минус.

In [28]:
print(f'Достаточный объем сырья для безубыточной разработки: {round(cost_per_well / REVENUE_PER_BARREL, 2)} тыс. баррелей')

Достаточный объем сырья для безубыточной разработки: 111.11 тыс. баррелей


Теперь посмотрим, каков средний запас нефти в скважинах в каждом из регионов.

In [29]:
print(f"Средний запас нефти (в тыс. баррелей) в каждом регионе {geo_data_0['product'].mean(), geo_data_1['product'].mean(), geo_data_2['product'].mean()}")

Средний запас нефти (в тыс. баррелей) в каждом регионе (92.49948184460142, 68.82523184270339, 95.00042479767433)


### Вывод

Для безубыточной разработки с одной скважины нужно добыть минимум 111.11 тыс. баррелей. Среднее же по регионам составляет 92, 69 и 95 тыс. баррелей соответсвенно. Поэтому для безубыточного производста произвольные скважины нам не подойдут, нужно отбирать самые лучшие.

## Расчёт прибыли и рисков 

### Расчет прибыли

Отберем по 200 лучших скважин из каждого региона, и посмотрим среднее значение объема нефти.

In [30]:
best_wells_0 = pd.Series(y_pred_0).sort_values(ascending=False)[:200]
best_wells_1 = pd.Series(y_pred_1).sort_values(ascending=False)[:200]
best_wells_2 = pd.Series(y_pred_2).sort_values(ascending=False)[:200]
best_wells_0.mean()
print(f'Средний объем 200 лучших скважин 1 региона {round(best_wells_0.mean(), 2)} тыс. баррелей')
print(f'Средний объем 200 лучших скважин 2 региона {round(best_wells_1.mean(), 2)} тыс. баррелей')
print(f'Средний объем 200 лучших скважин 3 региона {round(best_wells_2.mean(), 2)} тыс. баррелей')

Средний объем 200 лучших скважин 1 региона 161.6 тыс. баррелей
Средний объем 200 лучших скважин 2 региона 139.09 тыс. баррелей
Средний объем 200 лучших скважин 3 региона 153.62 тыс. баррелей


Получается, что если брать 200 лучших скважин из каждого региона, то все они принесут прибыль, так как их средний объем привышает 111.11 тыс. баррелей. Теперь расчитаем суммарный объем этих скважин в каждом из регионов.

In [31]:
print(f'Суммарный объем 200 лучших скважин 1 региона {round(best_wells_0.sum(), 2)} тыс. баррелей')
print(f'Суммарный объем 200 лучших скважин 2 региона {round(best_wells_1.sum(), 2)} тыс. баррелей')
print(f'Суммарный объем 200 лучших скважин 3 региона {round(best_wells_2.sum(), 2)} тыс. баррелей')

Суммарный объем 200 лучших скважин 1 региона 32319.6 тыс. баррелей
Суммарный объем 200 лучших скважин 2 региона 27817.99 тыс. баррелей
Суммарный объем 200 лучших скважин 3 региона 30723.94 тыс. баррелей


Теперь рассчитаем, какую прибыль нам принесет каждый регион. Для этого посчитаем доходы и расходы. Расходы - это 10 млрд. рублей на разработку. А доходы - это вся вырученная нефть по стоимости в 450 тыс. рублей за единицу измерения.

In [32]:
profit_0 = best_wells_0.sum() * REVENUE_PER_BARREL - BUDGET
profit_1 = best_wells_1.sum() * REVENUE_PER_BARREL - BUDGET
profit_2 = best_wells_2.sum() * REVENUE_PER_BARREL - BUDGET
print(f'Возможная прибыль в 1 регионе: {round(profit_0 / 1000000, 2)} млн. рублей')
print(f'Возможная прибыль во 2 регионе: {round(profit_1 / 1000000, 2)} млн. рублей')
print(f'Возможная прибыль в 3 регионе: {round(profit_2 / 1000000, 2)} млн. рублей')

Возможная прибыль в 1 регионе: 4543.82 млн. рублей
Возможная прибыль во 2 регионе: 2518.09 млн. рублей
Возможная прибыль в 3 регионе: 3825.77 млн. рублей


### Расчет рисков

Напишем функцию для расчета распределения прибыли. Мы можем взять на анализ 500 точек и выбрать из них 200 лучших.
Будем использовать bootstrap с 1000 повторений для расчетов. Каждый раз мы будем брать 500 точек, находить 200 лучших (по предсказанным значениям) и считать нашу прибыль (но уже по заведомо известным значениям). И потом посмотрим, какая средняя прибыль получилась. А также найдем 95% доверительный интервал и риск убытков. Риск убытков - это отношение количества исходов с отрицательной прибылью к общему количеству исходов (1000 шт.), выраженное в процентах. С помощью функции исследуем все 3 региона.

In [33]:
def profit(df_pred):   
    state = np.random.RandomState(12345)
    values = []
    loss = 0
    for i in range(1000):
        #создаем выборку предсказанных значений размером N_WELLS
        subsample = df_pred['y_pred'].sample(n=N_WELLS, replace=True, random_state=state)
        #сортируем значения по убыванию и берем 200 самых больших значений
        subsample_200 = subsample.sort_values(ascending=False)[:200]
        #находим настоящие значения выбраных скважин. для этого берем индексы предсказаний и соотносим их с реальными значениями.
        true_values = df_pred['product'][subsample_200.index]
        #считаем прибыль
        revenue = true_values.sum() * REVENUE_PER_BARREL - BUDGET
        values.append(revenue / 1000000)

    values = pd.Series(values)
    loss = len(values[values < 0]) / len(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)

    mean = values.mean()
    print(f'Средняя выручка: {round(mean, 2)} млн. рублей')
    print(f'95%-й доверительный интервал: от {round(lower, 2)} млн. рублей до {round(higher, 2)} млн. рублей')
    print(f'Риск убытков: {round(loss*100, 2)}%')

Применим функцию к каждому из регионов

In [34]:
print('1 регион')
print()
profit(df_pred_0)

1 регион

Средняя выручка: 409.08 млн. рублей
95%-й доверительный интервал: от -134.18 млн. рублей до 887.57 млн. рублей
Риск убытков: 5.9%


In [35]:
print('2 регион')
print()
profit(df_pred_1)

2 регион

Средняя выручка: 454.6 млн. рублей
95%-й доверительный интервал: от 48.18 млн. рублей до 843.8 млн. рублей
Риск убытков: 1.1%


In [36]:
print('3 регион')
print()
profit(df_pred_2)

3 регион

Средняя выручка: 381.19 млн. рублей
95%-й доверительный интервал: от -136.45 млн. рублей до 908.52 млн. рублей
Риск убытков: 7.9%


### Вывод

Несмотря на наименьший средний объем нефти в скважинах (около 69 тыс. баррелей) во 2 регионе, наибольшая средняя выручка получается именно в нем. Она составляет 454.6 млн. рублей. При этом вероятность получить убытки составляет 1.1%. 

Наименьшая средняя выручка получилась в 3 регионе, она составляет 381.19 млн. рублей. А риск убытков - 7.9%.

В 1 регионе средняя выручка равна 409.08 млн. рублей, а риск убытков 5.9%.

Мне кажется, что для разработки нужно использовать 2 регион. В нем самая большая средняя выручка, при этом самый низкий риск убытков в 5-8 раз меньше, чем в других регионах. Верхняя граница доверительного интервала у него самая низкая из всех регионов, но при этом нижняя граница - положительная в отличии от других регионов. То есть это регион самый безопасный, что касается рисков, при этом по прибыли он самый высокий.